In [ ]:
import requests
import json
import time
import stomp
from datetime import datetime

# Reddit API
CLIENT_ID = 'Z03wJZXTyb5beGqyVLxXdQ'
SECRET_KEY = '9NVOxG5n-FWHpLjwOpUgPd9QdzH-Tw'
USERNAME = 'Shjsuii'
PASSWORD = 'Nghia123@'

# ActiveMQ server
ACTIVE_MQ_HOST = 'localhost'
ACTIVE_MQ_PORT = 61613
QUEUE_NAME_THREADS = '/queue/reddit_threads'
QUEUE_NAME_COMMENTS = '/queue/reddit_comments'

# Authenticate Reddit
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
data = {'grant_type': 'password', 'username': USERNAME, 'password': PASSWORD}
#User-Agent': 'RedditStream/0.1
headers = {'User-Agent': 'RedditStream/0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
token = res.json()['access_token']
headers['Authorization'] = f'bearer {token}'

# Connect to ActiveMQ
conn = stomp.Connection([(ACTIVE_MQ_HOST, ACTIVE_MQ_PORT)])
conn.connect(wait=True)

def fetch_and_publish():
    after_thread = None  # ID bài viết cuối cùng đã lấy
    after_comment = None  # ID bình luận cuối cùng đã lấy

    while True:
        try:
            new_data_found = False  # Biến kiểm tra có dữ liệu mới hay không

            # Fetch subreddit threads
            url_threads = f'https://oauth.reddit.com/r/VietNam/new.json?limit=100'
            if after_thread:
                url_threads += f"&after={after_thread}"

            response_threads = requests.get(url_threads, headers=headers)
            threads = response_threads.json().get('data', {}).get('children', [])

            if threads:
                new_data_found = True
                for thread in threads:
                    t = thread['data']
                    thread_data = {
                        'thread_id': t['id'],
                        'author': t['author'],
                        'title': t['title'],
                        'selftext': t['selftext'],
                        'link_flair_text': t.get('link_flair_text'),
                        'num_comments': t['num_comments'],
                        'score': t['score'],
                        'upvote_ratio': t['upvote_ratio'],
                        'over_18': t['over_18'],
                        'is_self': t['is_self'],
                        'distinguished': t.get('distinguished'),
                        'stickied': t['stickied'],
                        'locked': t['locked'],
                        'external_url': t['url'],
                        'created_utc': datetime.utcfromtimestamp(t['created_utc']).isoformat()
                    }
                    conn.send(QUEUE_NAME_THREADS, json.dumps(thread_data))
                    print(f'Sent thread to ActiveMQ: {thread_data}')

                after_thread = threads[-1]['data']['name']

            # Fetch subreddit comments
            url_comments = f'https://oauth.reddit.com/r/VietNam/comments.json?limit=100'
            if after_comment:
                url_comments += f"&after={after_comment}"

            response_comments = requests.get(url_comments, headers=headers)
            comments = response_comments.json().get('data', {}).get('children', [])

            if comments:
                new_data_found = True
                for comment in comments:
                    c = comment['data']
                    comment_data = {
                        'comment_id': c['id'],
                        'thread_id': c['link_id'].split('_')[-1],
                        'parent_id': c['parent_id'],
                        'author': c['author'],
                        'body': c['body'],
                        'score': c['score'],
                        'is_submitter': c['is_submitter'],
                        'distinguished': c.get('distinguished'),
                        'stickied': c['stickied'],
                        'created_utc': datetime.utcfromtimestamp(c['created_utc']).isoformat()
                    }
                    conn.send(QUEUE_NAME_COMMENTS, json.dumps(comment_data))
                    print(f'Sent comment to ActiveMQ: {comment_data}')

                after_comment = comments[-1]['data']['name']

            if not new_data_found:
                print("------------Không có dữ liệu mới, chờ 10 giây rồi thử lại...------------")

            time.sleep(10)  # Chờ 10 giây trước khi fetch tiếp

        except Exception as e:
            print(f'------------Error fetching data: {e} ------------')
            time.sleep(30) 
if __name__ == '__main__':
    fetch_and_publish()


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_31576\4266150320.py:68: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'created_utc': datetime.utcfromtimestamp(t['created_utc']).isoformat()


Sent thread to ActiveMQ: {'thread_id': '1iykcql', 'author': 'BestPotatoEverr', 'title': 'Context of using lam on / vui long cho, and lam on / cam on', 'selftext': 'Learning Vietnamese (on Duolingo) and there seems to be several words for the same word (eg, please and thank you). \n\nWhat’s would be there difference in context and use please?\n\nPlease - lam on / vui long cho \n\nThank you - lam on / cam on ', 'link_flair_text': 'Discussion/Thảo luận', 'num_comments': 0, 'score': 1, 'upvote_ratio': 1.0, 'over_18': False, 'is_self': True, 'distinguished': None, 'stickied': False, 'locked': False, 'external_url': 'https://www.reddit.com/r/VietNam/comments/1iykcql/context_of_using_lam_on_vui_long_cho_and_lam_on/', 'created_utc': '2025-02-26T10:16:37'}
Sent thread to ActiveMQ: {'thread_id': '1iykaxx', 'author': 'PurpleMorning321', 'title': 'Scooter Rental from Saigon to Da Nang?', 'selftext': 'Hello,\n\nme and my friend will be in southern/central vietnam for 3 weeks, starting next week. We

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_31576\4266150320.py:97: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'created_utc': datetime.utcfromtimestamp(c['created_utc']).isoformat()


Sent comment to ActiveMQ: {'comment_id': 'mev22jo', 'thread_id': '1iyd47e', 'parent_id': 't1_metlegh', 'author': 'rootsoap', 'body': 'In Finland the most common words are "made in China"', 'score': 1, 'is_submitter': False, 'distinguished': None, 'stickied': False, 'created_utc': '2025-02-26T10:09:41'}
Sent comment to ActiveMQ: {'comment_id': 'mev228u', 'thread_id': '1iw3kz6', 'parent_id': 't1_melo9fh', 'author': 'stankar1990', 'body': "\\^\\^ this, however Asian culture doesn't understand this yet", 'score': 1, 'is_submitter': False, 'distinguished': None, 'stickied': False, 'created_utc': '2025-02-26T10:09:36'}
Sent comment to ActiveMQ: {'comment_id': 'mev1scj', 'thread_id': '1iwnorh', 'parent_id': 't1_mehv4ld', 'author': 'HolyMopOfCheese', 'body': "In Vietnamese history, we have never ever achieved democracy, whether it be the north, the south, or even the current gov   \nNorth Vietnam was a one-party state, that's true \n\n  \nSouth Vietnam appeared to be multi-party, but there was